In [1]:
from youper import (get_data, ReflectionModel, generate_reflection_from_tensors, 
                    observe_reflection, generate_reflection_from_string)

from transformers import RobertaModel, RobertaTokenizer
import torch

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
df = get_data()
df.shape

(2271, 14)

In [4]:
df.head()

,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views,split,first_an_sent,seems_sounds_sents,reflection
0,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Sherry Katz, LCSWCouples and Family Therapist,...",https://counselchat.com/therapists/sherry-katz...,"If everyone thinks you're worthless, then mayb...",1,2899,train,"If everyone thinks you're worthless, then mayb...",None,"If everyone thinks you're worthless, then mayb..."
1,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Robin Landwehr, DBH, LPCC, NCCMental Health in...",https://counselchat.com/therapists/robin-landw...,"Hello, and thank you for your question and see...",1,3514,train,"Hello, and thank you for your question and see...",None,"Hello, and thank you for your question and see..."
2,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,Lee KingI use an integrative approach to treat...,https://counselchat.com/therapists/lee-king,First thing I'd suggest is getting the sleep y...,0,5,train,First thing I'd suggest is getting the sleep y...,None,First thing I'd suggest is getting the sleep y...
3,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Shauntai Davis-YearginPersonalized, private on...",https://counselchat.com/therapists/shauntai-da...,Therapy is essential for those that are feelin...,0,31,train,Therapy is essential for those that are feelin...,None,Therapy is essential for those that are feelin...
4,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,Jordan WhiteLicensed Social Worker at Oak Root...,https://counselchat.com/therapists/jordan-white,I first want to let you know that you are not ...,0,620,train,I first want to let you know that you are not ...,None,I first want to let you know that you are not ...


## Train / Valid / Test Split

In [5]:
train_mask = df.split == 'train' 
valid_mask = df.split == 'val'
test_mask = df.split == 'test'
train_mask.sum(), valid_mask.sum(), test_mask.sum()

(1963, 185, 123)

## Pretrained RoBERTA encoder

In [6]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta = RobertaModel.from_pretrained('roberta-base')

## Reflection model

In [7]:
addl_state = {
    'df': df,
    'train_mask': train_mask,
    'valid_mask': valid_mask,
    'tokenizer': tokenizer}

model = ReflectionModel(roberta, addl_state=addl_state)

In [8]:
model.load_state_dict(torch.load('models/reflections.pt'))

<All keys matched successfully>

## GENERATE REFLECTIONS

In [9]:
start_token_id = tokenizer.convert_tokens_to_ids('<s>')
end_token_id = tokenizer.convert_tokens_to_ids('</s>')
start_token_id, end_token_id

(0, 2)

In [10]:
%%time

def gen_reflection(s):
    pred_toks = generate_reflection_from_string(model, tokenizer, s, start_token_id, end_token_id)
    return tokenizer.decode(pred_toks)

df['gen_reflection'] = df.questionText.apply(gen_reflection)

CPU times: user 2min 31s, sys: 21.9 s, total: 2min 53s
Wall time: 2min 53s


## Observe some generated reflections

In [11]:
def show_example(df):
    sample = df.sample(1)
    print('='*70)

    print('QUESTION:')
    print(sample['questionText'].iloc[0])

    print('\nACTUAL REFLECTION:')
    print(sample['reflection'].iloc[0])

    print('\nGENERATED REFLECTION:')
    print(sample['gen_reflection'].iloc[0])
    print()

### Training set

In [17]:
for _ in range(5): show_example(df[train_mask])

QUESTION:
How does a person start the counseling process?

ACTUAL REFLECTION:
Hello.

GENERATED REFLECTION:
 I would recommend that you have a good fit for you to find a counselor.

QUESTION:
In middle school and high school, my friends and family thought I was gay. I tried telling them, but they wouldn’t believe me. It almost feels like they wanted me to be. Now I’m actually starting to believe them. I know I wasn’t back then, and now I’m not sure anymore.

ACTUAL REFLECTION:
I agree with Amy.

GENERATED REFLECTION:
 I am sorry to hear about your question.

QUESTION:
I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?

ACTUAL REFLECTION:
Therapy is essential for those that are feeling depressed and worthless.

GE

### Validation set

In [25]:
for _ in range(5): show_example(df[valid_mask])

QUESTION:
My grandma had a stroke and passed away recently. I lost my home and job. I'm looking but haven't found a job. I've been binge watching television and binge eating.

ACTUAL REFLECTION:
Wow, you got hit with some serious stuff all at once.

GENERATED REFLECTION:
 I'm sorry to hear about your question.

QUESTION:
My boyfriend and I have been dating for almost a year. We go to different schools, and we don't see each other that often. I just feel very paranoid that there's someone else or that he could be hiding something from me.

ACTUAL REFLECTION:
Paranoid is a mental disorder word.

GENERATED REFLECTION:
 I am sorry to you're feeling about your boyfriend.

QUESTION:
I told her that if daddy dies, we will never see him again. She started crying because I wouldn't make her daddy die.

ACTUAL REFLECTION:
Children often have a difficult time expressing emotions accurately.

GENERATED REFLECTION:
 I appreciate that you have a good question.

QUESTION:
I’m trying to make marriage 

In [65]:
prompt = """I'm annoyed because this COVID quarantine is isolating me from my friends"""
observe_reflection(model, tokenizer, string=prompt)

Question:
<s> I'm annoyed because this stupid COVID quarantine is isolating me from my friends</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [66]:
prompt = """I'm having trouble sleeping and it's ruining a lot of my days"""
observe_reflection(model, tokenizer, string=prompt)

Question:
<s> I'm having trouble sleeping and it's ruining a lot of my days</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [67]:
prompt = """I'm depressed because I'm a terrible video game player and my friend Adam is sooo much better"""
observe_reflection(model, tokenizer, string=prompt)

Question:
<s> I'm depressed because I'm a terrible video game player and my friend Adam is sooo much better</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [68]:
prompt = """My girlfriend left me, but it's actually making me happy"""
observe_reflection(model, tokenizer, string=prompt)

Question:
<s> My girlfriend left me, but it's actually making me happy</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [69]:
prompt = """I really don't like being forced to work over the weekend"""
observe_reflection(model, tokenizer, string=prompt)

Question:
<s> I really don't like being forced to work over the weekend</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [72]:
torch.save(model.state_dict(), 'models/reflections.pt')